In [2]:
!pip install psycopg2

In [4]:
!pip install include

In [93]:
import os
import pandas as pd
import sys
import psycopg2
from pprint import pprint
import MySQLdb
import pymysql
pymysql.install_as_MySQLdb()
import mysql.connector as msql
from mysql.connector import Error
# Import internal snippets     
#from include.db_config import *
#from include.MySQLCursorDict import *

# Open database connections   
# Mysql connection
try:
    conn_mysql= msql.connect(host='localhost', database='davis2', user='root', password='bo2108luwatife@1')
    print('Connected to MySQL Database')
except Error as e:
    print("Error while connecting to MySQL", e)
    sys.exit(1)
# Postgresql connection
try:
    conn_psql = psycopg2.connect(host='localhost', database='sensor_data', user='postgres', password='bo2108luwatife@1')
    print('Connected to PostgreSQL Database')
except psycopg2.Error as e:
    print('PSQL: Unable to connect!\n{0}').format(e)
    sys.exit(1)
    
 # Cursors initializations
cur_mysql = conn_mysql.cursor(dictionary=True)
cur_psql = conn_psql.cursor()

 #create table
mysql_query = pd.read_sql_query('''SELECT flow1, occupancy1, flow2, 
                    flow3, occupancy3 FROM davis2.load_data3''', conn_mysql)
new_df = pd.DataFrame(mysql_query)

create_psql_table = '''CREATE TABLE load_data3 (
                        flow1 FLOAT, occupancy1 FLOAT, flow2 FLOAT, flow3 FLOAT, occupancy3 FLOAT)'''
cur_psql.execute(create_psql_table)

for i, row in new_df.iterrows():
    try:
        psql = 'INSERT INTO first_data.load_data3 VALUES (%s, %s, %s, %s, %s, %s)'
        cur_psql.execute(psql, tuple(row))
        
    except psycopg2.Error as e:
        print("cannot execute that query!", e.pgerror)
        sys.exit("Some problem occured with that query!")   
## Closing cursors
cur_mysql.close()
cur_psql.close()

## Committing 
conn_psql.commit()

## Closing database connections
conn_mysql.close()
conn_psql.close()

Connected to MySQL Database
Connected to PostgreSQL Database


IndexError: index 5 is out of bounds for axis 0 with size 5

In [92]:
print(row)

flow1               2
occupancy1     0.0122
flow2             3.0
flow3         0.01945
occupancy3     0.0122
Name: 0, dtype: object
